<img align="center" src="https://iili.io/3wI8gI.png" style="height:90px" style="width:30px"/>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from twilio.rest import Client

In [13]:
url = "https://www.ica.se/erbjudanden/ica-supermarket-medborgarplatsen-1003941/"
client = OpenAI()
account_sid = 'your_account_sd'
auth_token = 'your_auth_token'
my_twilio_number = "+16413299119"
my_phone_number = "+46702398619"
client_twilio = Client(account_sid, auth_token)

# Describe the role and give instructions to ChatGPT
role = "Du är en kreativ kock som är väldigt duktig på att rekommendera enkla maträtter baserat på tillgängliga ingredienser."
instructions = "Jag vill att du rekommenderar mig maträtter som jag kan laga till middag och sedan göra matlåda av. Du får en lista med veckans rabatterade priser från en matbutik. Du ska använda listan för att skapa 3 enkla recept. Du får också lägga till basvaror (pasta, ris, potatis, lök, couscous, ägg, billigare grönsaker, etc.), även om de inte är med på listan.\nJag vill att du genererar 3 rätter i rubrikform och under varje rubrik/rätt så ska det finnas en lista med ingredienser för 4 portioner. Det är viktigt att enheterna är i enlighet med det metriska systemet. Under listan med ingredienser ska du beskriva väldigt kort hur man tillagar rätten."


def scrape_your_local_ica(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)

    # find and click cookie button"
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver, 3)
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector)
    element.click()
    # scroll to bottoom of page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # generate a soup
    soup = BeautifulSoup(driver.page_source, "lxml")
    key = "article > div:nth-child(3)"
    containers = soup.select(key)
    offers = [i.text for i in containers]
    return offers


def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": instructions},
                        {"role": "user", "content": prompt}
                    ])
    return completion

def send_text(from_number, to_number, text_message):
    message = client_twilio.messages.create(from_=from_number, to=to_number, body=text_message)
    
# scrape and retrieve offers from local Ica store
context = scrape_your_local_ica(url)
#pass cheap products alongside instructions to chat gpt func
completion = chat_gpt_bot(client, role, instructions+"\n"+str(context))
# generate shopping list    
shopping_lst = chat_gpt_bot(client, "Hjälpreda", "Generera en inköpslista i punktform med sammanlagda enheter av följande information: "+completion.choices[0].message.content)
# send shopping list by text
send_text(my_twilio_number, my_phone_number, shopping_lst.choices[0].message.content)
# write a txt file
with open("weekly_recipe.txt", "w") as file:
    file.write(completion.choices[0].message.content)
                               